In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pyraug
!git clone https://github.com/clementchadebec/pyraug.git
import os
path=os.path.join(os.getcwd(), 'pyraug/examples')
os.chdir(path)

fatal: destination path 'pyraug' already exists and is not an empty directory.


In [3]:
pip install scikit-image

In [4]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import torch
from pyraug.trainers.training_config import TrainingConfig
from pyraug.pipelines.training import TrainingPipeline
from pyraug.models.rhvae import RHVAEConfig
from pyraug.models import RHVAE
from pyraug.pipelines.generation import GenerationPipeline
from pyraug.models.nn import BaseEncoder, BaseDecoder
import torch.nn as nn
import random
import numpy as np
from skimage.metrics import structural_similarity as compare_ssim

# Data

In [5]:
file_list = os.listdir('/content/drive/MyDrive/MSc data/data-4-genam-202209')
array_list = []
for file_name in file_list:
  file_path = os.path.join('/content/drive/MyDrive/MSc data/data-4-genam-202209', file_name)
  df = pd.read_csv(file_path)
  x_values = df['Points:0'].unique()
  y_values = df['Points:1'].unique()
  z_values = df['Points:2'].unique()
  x_len = len(x_values)
  y_len = len(y_values)
  z_len = len(z_values)
  pivot_df = df.pivot_table(index='Points:2', columns=['Points:1', 'Points:0'], values='pabs')
  array_3d = np.array([pivot_df[col].values for col in pivot_df.columns]).reshape(x_len, y_len, z_len)
  array_list.append(array_3d)
real_data_4_amp = np.stack(array_list)

In [6]:
file_list = os.listdir('/content/drive/MyDrive/MSc data/data-8-genam-202209')
array_list = []
for file_name in file_list:
  file_path = os.path.join('/content/drive/MyDrive/MSc data/data-8-genam-202209', file_name)
  df = pd.read_csv(file_path)
  x_values = df['Points:0'].unique()
  y_values = df['Points:1'].unique()
  z_values = df['Points:2'].unique()
  x_len = len(x_values)
  y_len = len(y_values)
  z_len = len(z_values)
  pivot_df = df.pivot_table(index='Points:2', columns=['Points:1', 'Points:0'], values='pabs')
  array_3d = np.array([pivot_df[col].values for col in pivot_df.columns]).reshape(x_len, y_len, z_len)
  array_list.append(array_3d)
real_data_8_amp = np.stack(array_list)

In [7]:
file_list = os.listdir('/content/drive/MyDrive/MSc data/data-16-genam-202210')
array_list = []
for file_name in file_list:
  file_path = os.path.join('/content/drive/MyDrive/MSc data/data-16-genam-202210', file_name)
  df = pd.read_csv(file_path)
  x_values = df['Points:0'].unique()
  y_values = df['Points:1'].unique()
  z_values = df['Points:2'].unique()
  x_len = len(x_values)
  y_len = len(y_values)
  z_len = len(z_values)
  pivot_df = df.pivot_table(index='Points:2', columns=['Points:1', 'Points:0'], values='pabs')
  array_3d = np.array([pivot_df[col].values for col in pivot_df.columns]).reshape(x_len, y_len, z_len)
  array_list.append(array_3d)
real_data_16_amp = np.stack(array_list)

In [8]:
file_list = os.listdir('/content/drive/MyDrive/MSc data/data-32-genam-202210')
array_list = []
for file_name in file_list:
  file_path = os.path.join('/content/drive/MyDrive/MSc data/data-32-genam-202210', file_name)
  df = pd.read_csv(file_path)
  x_values = df['Points:0'].unique()
  y_values = df['Points:1'].unique()
  z_values = df['Points:2'].unique()
  x_len = len(x_values)
  y_len = len(y_values)
  z_len = len(z_values)
  pivot_df = df.pivot_table(index='Points:2', columns=['Points:1', 'Points:0'], values='pabs')
  array_3d = np.array([pivot_df[col].values for col in pivot_df.columns]).reshape(x_len, y_len, z_len)
  array_list.append(array_3d)
real_data_32_amp = np.stack(array_list)

In [9]:
real_data_4_amp = torch.tensor(real_data_4_amp, dtype=torch.float32)
real_data_8_amp = torch.tensor(real_data_8_amp, dtype=torch.float32)
real_data_16_amp = torch.tensor(real_data_16_amp, dtype=torch.float32)
real_data_32_amp = torch.tensor(real_data_32_amp, dtype=torch.float32)

In [10]:
data_min = torch.min(real_data_4_amp)
data_max = torch.max(real_data_4_amp)
real_data_4_amp_normalized = (real_data_4_amp - data_min) / (data_max - data_min)
data_min = torch.min(real_data_8_amp)
data_max = torch.max(real_data_8_amp)
real_data_8_amp_normalized = (real_data_8_amp - data_min) / (data_max - data_min)
data_min = torch.min(real_data_16_amp)
data_max = torch.max(real_data_16_amp)
real_data_16_amp_normalized = (real_data_16_amp - data_min) / (data_max - data_min)
data_min = torch.min(real_data_32_amp)
data_max = torch.max(real_data_32_amp)
real_data_32_amp_normalized = (real_data_32_amp - data_min) / (data_max - data_min)

In [11]:
real_data_4_amp_normalized = real_data_4_amp_normalized.view(1024,100)
real_data_8_amp_normalized = real_data_8_amp_normalized.view(1024,567)
real_data_16_amp_normalized = real_data_16_amp_normalized.view(1024,3468)
real_data_32_amp_normalized = real_data_32_amp_normalized.view(1024,25047)

In [12]:
random_indices = torch.randperm(1024)
real_data_4_amp_normalized = real_data_4_amp_normalized[random_indices]
training_4_amp = real_data_4_amp_normalized[:900, :]
test_4_amp = real_data_4_amp_normalized[900:, :]

In [13]:
random_indices = torch.randperm(1024)
real_data_8_amp_normalized = real_data_8_amp_normalized[random_indices]
training_8_amp = real_data_8_amp_normalized[:900, :]
test_8_amp = real_data_8_amp_normalized[900:, :]

In [14]:
random_indices = torch.randperm(1024)
real_data_16_amp_normalized = real_data_16_amp_normalized[random_indices]
training_16_amp = real_data_16_amp_normalized[:900, :]
test_16_amp = real_data_16_amp_normalized[900:, :]

In [15]:
random_indices = torch.randperm(1024)
real_data_32_amp_normalized = real_data_32_amp_normalized[random_indices]
training_32_amp = real_data_32_amp_normalized[:900, :]
test_32_amp = real_data_32_amp_normalized[900:, :]

In [16]:
def compute_cosine_similarity(array1, array2):
    if array1.shape != array2.shape:
        raise ValueError("different size")
    vector1 = array1.reshape(-1)
    vector2 = array2.reshape(-1)

    dot_product = np.dot(vector1, vector2)

    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)

    cosine_similarity = dot_product / (norm1 * norm2)

    return cosine_similarity

In [17]:
def compute_ssim_3d(array1, array2, win_size=3):
    if array1.shape != array2.shape:
        raise ValueError("different size")

    ssim_values = []

    for i in range(array1.shape[0]):
        slice1 = array1[i, :, :]
        slice2 = array2[i, :, :]
        ssim_score, _ = compare_ssim(slice1, slice2, win_size=3, multichannel=False, full=True)
        ssim_values.append(ssim_score)

    mean_ssim = np.mean(ssim_values)
    return mean_ssim

# MLP

In [18]:
class Encoder_Conv(BaseEncoder):
    def __init__(self, args):
        BaseEncoder.__init__(self)
        self.latent_dim = args.latent_dim

        self.layers = nn.Sequential(
            nn.Linear(100, 30),
            nn.ReLU(),
            nn.Linear(30, 30),
            nn.ReLU(),
        )


        self.mu = nn.Linear(30, self.latent_dim)
        self.std = nn.Linear(30, self.latent_dim)

    def forward(self, x):
        out = self.layers(x)
        return self.mu(out), self.std(out)


class Decoder_Conv(BaseDecoder):
    def __init__(self, args):
        BaseDecoder.__init__(self)

        self.latent_dim = args.latent_dim

        self.fc1 = nn.Sequential(
            nn.Linear(self.latent_dim, 30),
            nn.ReLU(),
            nn.Linear(30, 50),
            nn.ReLU(),
            nn.Linear(50, 100),
            nn.Sigmoid()
        )


    def forward(self, z):
        out = self.fc1(z)
        return out

In [19]:
model_config = RHVAEConfig(
    input_dim=5*5*4, # This is needed since we do not provide any encoder, decoder and metric architecture
    latent_dim=10)

In [20]:
encoder = Encoder_Conv(model_config)
decoder= Decoder_Conv(model_config)

In [21]:
model = RHVAE(
    model_config=model_config,
    encoder=encoder,
    decoder=decoder
)

In [22]:
training_config = TrainingConfig(
    output_dir='my_model_with_custom_archi',
    no_cuda=False,
    learning_rate=1e-3,
    batch_size=50,
    train_early_stopping=100,
    steps_saving=None,
    max_epochs=200)

In [23]:
pipeline = TrainingPipeline(
    data_loader=None,
    data_processor=None,
    model=model,
    optimizer=None,
    training_config=training_config)

In [24]:
torch.manual_seed(7)
torch.cuda.manual_seed(7)

pipeline(
    train_data=training_4_amp,
    eval_data=training_4_amp,
    log_output_dir='output_logs'
)

Data normalized using individual_min_max_scaling.
INFO:pyraug.data.preprocessors:Data normalized using individual_min_max_scaling.
 -> If this is not the desired behavior pass an instance of DataProcess with 'data_normalization_type' attribute set to desired normalization or None

INFO:pyraug.data.preprocessors: -> If this is not the desired behavior pass an instance of DataProcess with 'data_normalization_type' attribute set to desired normalization or None

Data normalized using individual_min_max_scaling.
INFO:pyraug.data.preprocessors:Data normalized using individual_min_max_scaling.
 -> If this is not the desired behavior pass an instance of DataProcess with 'data_normalization_type' attribute set to desired normalization or None

INFO:pyraug.data.preprocessors: -> If this is not the desired behavior pass an instance of DataProcess with 'data_normalization_type' attribute set to desired normalization or None

Created my_model_with_custom_archi folder since did not exist.

INFO:pyr

In [25]:
inputs = {"data": test_4_amp}
output= model(inputs)

In [26]:
reconstructed_x = output.recon_x.detach().numpy()

In [27]:
compute_cosine_similarity(reconstructed_x,test_4_amp)

0.6492033

In [28]:
reconstructed_x.shape

(124, 100)

In [29]:
reconstructed_x_ssim = reconstructed_x.reshape(124, 5, 5, 4)
test_4_amp_ssim = test_4_amp.view(124, 5, 5, 4).numpy()
ssim_values = []
for i in range(test_4_amp_ssim.shape[0]):
  ssim_score = compute_ssim_3d(test_4_amp_ssim[i], reconstructed_x_ssim[i], win_size=3)
  ssim_values.append([ssim_score])
ssim_values_mean = np.array(ssim_values).mean()

<ipython-input-17-3c500f7ede1b>:10: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_score, _ = compare_ssim(slice1, slice2, win_size=3, multichannel=False, full=True)


In [30]:
ssim_values_mean

0.15997847850006217

In [31]:
class Encoder_Conv(BaseEncoder):
    def __init__(self, args):
        BaseEncoder.__init__(self)
        self.latent_dim = args.latent_dim

        self.layers = nn.Sequential(
            nn.Linear(567, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
        )


        self.mu = nn.Linear(50, self.latent_dim)
        self.std = nn.Linear(50, self.latent_dim)

    def forward(self, x):
        out = self.layers(x)
        return self.mu(out), self.std(out)


class Decoder_Conv(BaseDecoder):
    def __init__(self, args):
        BaseDecoder.__init__(self)

        self.latent_dim = args.latent_dim

        self.fc1 = nn.Sequential(
            nn.Linear(self.latent_dim, 50),
            nn.ReLU(),
            nn.Linear(50, 100),
            nn.ReLU(),
            nn.Linear(100, 567),
            nn.Sigmoid()
        )


    def forward(self, z):
        out = self.fc1(z)
        return out

In [32]:
model_config = RHVAEConfig(
    input_dim=7*9*9, # This is needed since we do not provide any encoder, decoder and metric architecture
    latent_dim=10)

In [33]:
encoder = Encoder_Conv(model_config)
decoder= Decoder_Conv(model_config)

In [34]:
model = RHVAE(
    model_config=model_config,
    encoder=encoder,
    decoder=decoder
)

In [35]:
training_config = TrainingConfig(
    output_dir='my_model_with_custom_archi',
    no_cuda=False,
    learning_rate=1e-3,
    batch_size=50,
    train_early_stopping=100,
    steps_saving=None,
    max_epochs=200)

In [36]:
pipeline = TrainingPipeline(
    data_loader=None,
    data_processor=None,
    model=model,
    optimizer=None,
    training_config=training_config)

In [37]:
torch.manual_seed(7)
torch.cuda.manual_seed(7)

pipeline(
    train_data=training_8_amp,
    eval_data=training_8_amp,
    log_output_dir='output_logs'
)

Data normalized using individual_min_max_scaling.
INFO:pyraug.data.preprocessors:Data normalized using individual_min_max_scaling.
 -> If this is not the desired behavior pass an instance of DataProcess with 'data_normalization_type' attribute set to desired normalization or None

INFO:pyraug.data.preprocessors: -> If this is not the desired behavior pass an instance of DataProcess with 'data_normalization_type' attribute set to desired normalization or None

Data normalized using individual_min_max_scaling.
INFO:pyraug.data.preprocessors:Data normalized using individual_min_max_scaling.
 -> If this is not the desired behavior pass an instance of DataProcess with 'data_normalization_type' attribute set to desired normalization or None

INFO:pyraug.data.preprocessors: -> If this is not the desired behavior pass an instance of DataProcess with 'data_normalization_type' attribute set to desired normalization or None

Model passed sanity check !

INFO:pyraug.trainers.trainers:Model passed 

In [38]:
inputs = {"data": test_8_amp}
output= model(inputs)

In [39]:
reconstructed_x = output.recon_x.detach().numpy()

In [40]:
compute_cosine_similarity(reconstructed_x,test_8_amp)

0.6414106

In [41]:
reconstructed_x_ssim = reconstructed_x.reshape(124, 9, 9, 7)
test_8_amp_ssim = test_8_amp.view(124, 9, 9, 7).numpy()
ssim_values = []
for i in range(test_8_amp_ssim.shape[0]):
  ssim_score = compute_ssim_3d(test_8_amp_ssim[i], reconstructed_x_ssim[i], win_size=3)
  ssim_values.append([ssim_score])
ssim_values_mean = np.array(ssim_values).mean()

<ipython-input-17-3c500f7ede1b>:10: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_score, _ = compare_ssim(slice1, slice2, win_size=3, multichannel=False, full=True)


In [42]:
ssim_values_mean

0.19718702820536255

In [43]:
class Encoder_Conv(BaseEncoder):
    def __init__(self, args):
        BaseEncoder.__init__(self)
        self.latent_dim = args.latent_dim

        self.layers = nn.Sequential(
            nn.Linear(3468, 200),
            nn.ReLU(),
            nn.Linear(200, 50),
            nn.ReLU(),
        )


        self.mu = nn.Linear(50, self.latent_dim)
        self.std = nn.Linear(50, self.latent_dim)

    def forward(self, x):
        out = self.layers(x)
        return self.mu(out), self.std(out)


class Decoder_Conv(BaseDecoder):
    def __init__(self, args):
        BaseDecoder.__init__(self)

        self.latent_dim = args.latent_dim

        self.fc1 = nn.Sequential(
            nn.Linear(self.latent_dim, 50),
            nn.ReLU(),
            nn.Linear(50, 200),
            nn.ReLU(),
            nn.Linear(200, 3468),
            nn.Sigmoid()
        )


    def forward(self, z):
        out = self.fc1(z)
        return out

In [44]:
model_config = RHVAEConfig(
    input_dim=12*17*17, # This is needed since we do not provide any encoder, decoder and metric architecture
    latent_dim=15)

In [45]:
encoder = Encoder_Conv(model_config)
decoder= Decoder_Conv(model_config)

In [46]:
model = RHVAE(
    model_config=model_config,
    encoder=encoder,
    decoder=decoder
)

In [47]:
training_config = TrainingConfig(
    output_dir='my_model_with_custom_archi',
    no_cuda=False,
    learning_rate=1e-3,
    batch_size=50,
    train_early_stopping=100,
    steps_saving=None,
    max_epochs=200)

In [48]:
pipeline = TrainingPipeline(
    data_loader=None,
    data_processor=None,
    model=model,
    optimizer=None,
    training_config=training_config)

In [ ]:
torch.manual_seed(7)
torch.cuda.manual_seed(7)

pipeline(
    train_data=training_16_amp,
    eval_data=training_16_amp,
    log_output_dir='output_logs'
)

Data normalized using individual_min_max_scaling.
INFO:pyraug.data.preprocessors:Data normalized using individual_min_max_scaling.
 -> If this is not the desired behavior pass an instance of DataProcess with 'data_normalization_type' attribute set to desired normalization or None

INFO:pyraug.data.preprocessors: -> If this is not the desired behavior pass an instance of DataProcess with 'data_normalization_type' attribute set to desired normalization or None

Data normalized using individual_min_max_scaling.
INFO:pyraug.data.preprocessors:Data normalized using individual_min_max_scaling.
 -> If this is not the desired behavior pass an instance of DataProcess with 'data_normalization_type' attribute set to desired normalization or None

INFO:pyraug.data.preprocessors: -> If this is not the desired behavior pass an instance of DataProcess with 'data_normalization_type' attribute set to desired normalization or None

Model passed sanity check !

INFO:pyraug.trainers.trainers:Model passed 

In [ ]:
inputs = {"data": test_16_amp}
output= model(inputs)

In [ ]:
reconstructed_x = output.recon_x.detach().numpy()

In [ ]:
compute_cosine_similarity(reconstructed_x,test_16_amp)

0.59716296

In [ ]:
reconstructed_x_ssim = reconstructed_x.reshape(124, 17, 17, 12)
test_16_amp_ssim = test_16_amp.view(124, 17, 17, 12).numpy()
ssim_values = []
for i in range(test_16_amp_ssim.shape[0]):
  ssim_score = compute_ssim_3d(test_16_amp_ssim[i], reconstructed_x_ssim[i], win_size=3)
  ssim_values.append([ssim_score])
ssim_values_mean = np.array(ssim_values).mean()

<ipython-input-17-3c500f7ede1b>:10: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_score, _ = compare_ssim(slice1, slice2, win_size=3, multichannel=False, full=True)


In [ ]:
ssim_values_mean

0.27857571566601746